# 1 Metpy

MetPy是一个用于气象数据分析的开源项目。包括数据的读取、天气学相关计算以及诊断分析可视化。

安装： conda install -c conda-forge metpy

## 1.1 Metpy的单位制

MetPy在语法上的一大特点就是在传递给MetPy的函数之前，通常需要将单位属性附加给数组。

保证更高的计算准确性

不用再次进行单位的转换

In [1]:
from metpy.units import units

### 优点1：计算结果自带单位

In [5]:
x = np.arange(1,5)*units.m
t = units.Quantity(np.arange(1,5),'sec')
x/t

Magnitude,[1.0 1.0 1.0 1.0]
Units,meter/second


### 优点2：自动实现不同单位的转换

In [10]:
print(3*units.inch + 5*units.cm)
print((3*units.inch + 5*units.cm).to_base_units())

4.968503937007874 inch
0.1262 meter


## 1.2 Metpy的常量

In [11]:
import metpy.constants as constants

地球常数：

earth_avg_radius:地球平均半径(m)

earth_gravity:地球平均重力加速度(m s-2)

gravitational_constant:引力常数(m3 kg-1 s-2)

earth_mass:地球总质量(kg)

其它气象常数：

dry_air_gas_constant：地表干空气的气体常数(m s-2)

poisson_exponent:泊松方程中系数（Rd/Cp_d）

dry_adiabatic_lapse_rate:干绝热衰减率(K km-1)

In [12]:
print(5*units.m * constants.g)

49.033249999999995 meter ** 2 / second ** 2


## 1.3 举例

## 利用Metpy计算气象诊断量(以水汽通量散度为例)

$\bigtriangledown \cdot (Vq/g) = \frac{\partial }{\partial x}(uq/g)+\frac{\partial }{\partial y}(vq/g)$

In [13]:
import xarray as xr
from metpy import units
import metpy.constants as constants
import metpy.calc as mpcalc

In [16]:
f_u = xr.open_dataset('/home/mw/input/moyu1828/uwnd.mon.mean.nc')
u = f_u.uwnd.loc['2000-01-01',[1000,925,850,700,600,500,400,300]]

f_v = xr.open_dataset('/home/mw/input/moyu1828/vwnd.mon.mean.nc')
v = f_v.vwnd.loc['2000-01-01',[1000,925,850,700,600,500,400,300]]

f_q = xr.open_dataset('/home/mw/input/moyu1828/shum.mon.mean.nc')
q = f_q.shum.loc['2000-01-01',[1000,925,850,700,600,500,400,300]]

lat = q.lat
lon = q.lon
lev = q.level

In [17]:
qv_u = u*q/constants.g
qv_v = v*q/constants.g
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)

In [20]:
div_qv = np.zeros((lev.shape[0],lat.shape[0],lon.shape[0]))
for i in range(lev.shape[0]):
    div_qv[i] = mpcalc.divergence(u = qv_u[i],v = qv_v[i],dx = dx ,dy = dy)

In [29]:
import numpy as np
total_div_qv = np.trapz(div_qv[::-1],lev[::-1],axis=0)

In [28]:
help(np.trapz)

Help on function trapz in module numpy:

trapz(y, x=None, dx=1.0, axis=-1)
    Integrate along the given axis using the composite trapezoidal rule.
    
    Integrate `y` (`x`) along given axis.
    
    Parameters
    ----------
    y : array_like
        Input array to integrate.
    x : array_like, optional
        The sample points corresponding to the `y` values. If `x` is None,
        the sample points are assumed to be evenly spaced `dx` apart. The
        default is None.
    dx : scalar, optional
        The spacing between sample points when `x` is None. The default is 1.
    axis : int, optional
        The axis along which to integrate.
    
    Returns
    -------
    trapz : float
        Definite integral as approximated by trapezoidal rule.
    
    See Also
    --------
    sum, cumsum
    
    Notes
    -----
    Image [2]_ illustrates trapezoidal rule -- y-axis locations of points
    will be taken from `y` array, by default x-axis distances between
    points will 

In [18]:
help(mpcalc.divergence)

Help on function divergence in module metpy.calc.kinematics:

divergence(u, v, dx, dy)
    Calculate the horizontal divergence of a vector.
    
    Parameters
    ----------
    u : (M, N) `pint.Quantity`
        x component of the vector
    v : (M, N) `pint.Quantity`
        y component of the vector
    dx : `pint.Quantity`
        The grid spacing(s) in the x-direction. If an array, there should be one item less than
        the size of `u` along the applicable axis.
    dy : `pint.Quantity`
        The grid spacing(s) in the y-direction. If an array, there should be one item less than
        the size of `u` along the applicable axis.
    dim_order : str or ``None``, optional
        The ordering of dimensions in passed in arrays. Can be one of ``None``, ``'xy'``,
        or ``'yx'``. ``'xy'`` indicates that the dimension corresponding to x is the leading
        dimension, followed by y. ``'yx'`` indicates that x is the last dimension, preceded
        by y. ``None`` indicates t

### 利用Metpy计算大气视热源(一级简化热力方程)


![Image Name](https://cdn.kesci.com/upload/image/qxwv6lt84p.png?imageView2/0/w/960/h/960)


In [1]:
import xarray as xr 
import datetime as dt
import xarray as xr
from metpy.units import units
import metpy.constants as constants
import metpy.calc as mpcalc

f_u = xr.open_dataset('/home/mw/input/moyu1828/uwnd.mon.mean.nc')
u = f_u.uwnd.loc[f_u.time.dt.month.isin([7])].loc['1979-01-01':'2019-01-01',1000:100]
lat = u.lat
lon = u.lon
lev = u.level

f_v = xr.open_dataset('/home/mw/input/moyu1828/vwnd.mon.mean.nc')
v = f_v.vwnd.loc[f_v.time.dt.month.isin([7])].loc['1979-01-01':'2019-01-01',1000:100]

f_w = xr.open_dataset('/home/mw/input/moyu1828/omega.mon.mean.nc')
w = f_w.omega.loc[f_w.time.dt.month.isin([7])].loc['1979-01-01':'2019-01-01']

f_t = xr.open_dataset('/home/mw/input/moyu1828/air.mon.mean.nc')
t = f_t.air.loc[f_t.time.dt.month.isin([6,7,8])].loc['1979-01-01':'2019-01-01',1000:100]+273.15

In [2]:
u = np.array(u).mean((0))
v = np.array(v).mean((0))
w = np.array(w).mean((0))
t = np.array(t).reshape((3,40,12,73,144)).mean((1))
t_7 = t[1]

In [6]:
time = 31*24*60*60*units.s
#dTdt
dTdt = t[2]*units.K - t[0]*units.K
#advection
dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)
temadvect = np.zeros((12,73,144))
for i in range(12):
    temadvect[i,:,:] = mpcalc.advection(t_7[i], u[i], v[i],dx=dx, dy=dy,x_dim=-1, y_dim=-2)
temadvect = temadvect * units.K/units.s
#vertical
pressure = np.array(lev).reshape((12,1,1))*units.hPa
t_k = t_7*units.K
ss = (constants.dry_air_gas_constant* t_k)/constants.dry_air_spec_heat_press/pressure - np.gradient(t_k,axis=0)/np.gradient(pressure,axis=0)
ver = w*(units.Pa/units.s)*ss
#Q
q = ((dTdt - temadvect*time - ver*time)*constants.dry_air_spec_heat_press)/time
#total
total_q = np.trapz(q[::-1],lev[::-1],axis=0)

In [5]:
help(mpcalc.advection)

Help on function advection in module metpy.calc:

advection(scalar, u=None, v=None, w=None, *, dx=None, dy=None, dz=None, x_dim=-1, y_dim=-2, vertical_dim=-3)
    Calculate the advection of a scalar field by the wind.
    
    Parameters
    ----------
    scalar : `pint.Quantity` or `xarray.DataArray`
        Array (with N-dimensions) with the quantity to be advected. Use `xarray.DataArray` to
        have dimension ordering automatically determined, otherwise, use default
        [..., Z, Y, X] ordering or specify \*_dim keyword arguments.
    u, v, w : `pint.Quantity` or `xarray.DataArray` or None
        N-dimensional arrays with units of velocity representing the flow, with a component of
        the wind in each dimension. For 1D advection, use 1 positional argument (with `dx` for
        grid spacing and `x_dim` to specify axis if not the default of -1) or use 1 applicable
        keyword argument (u, v, or w) for proper physical dimension (with corresponding `d\*`
        for g

In [9]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cartopy.mpl.ticker as cticker

proj = ccrs.PlateCarree(central_longitude=0)
leftlon, rightlon, lowerlat, upperlat = (-180,180,-30,60)
img_extent = [leftlon, rightlon, lowerlat, upperlat]
fig = plt.figure(figsize=(12,8))
ax = fig.add_axes([0.1, 0.1, 0.5, 0.5],projection = proj)
ax.set_extent(img_extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m')) 
ax.set_xticks(np.array([-180,-120,-60,0,60,120,180]), crs=ccrs.PlateCarree())
ax.set_yticks(np.array([-30,0,30,60]), crs=ccrs.PlateCarree())
ax.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax.yaxis.set_major_formatter(cticker.LatitudeFormatter())
c1 = ax.contourf(lon,lat, total_q, zorder=0,levels =np.arange(-50,55,5) , 
                     extend = 'both', transform=ccrs.PlateCarree(), cmap=plt.cm.bwr)
position=fig.add_axes([0.18, 0.15,  0.35, 0.025])
fig.colorbar(c1,cax=position,orientation='horizontal',format='%.2f',)

Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.


/opt/conda/lib/python3.7/site-packages/cartopy/io/__init__.py:260: DownloadWarning: Downloading: https://naciscdn.org/naturalearth/50m/physical/ne_50m_coastline.zip
  warnings.warn('Downloading: {}'.format(url), DownloadWarning)


<Figure size 864x576 with 2 Axes>


![Image Name](https://cdn.kesci.com/upload/image/qxwwt713il.png?imageView2/0/w/960/h/960)
